In [ ]:
# This cell is just a reminder to ensure that the following extensions are loaded somewhere upstream in the pipeline.
%load_ext nbtools.r_support
%load_ext rpy2.ipython

In [ ]:
%%R

# The purpose of this cell is to load the Seurat library and install the R packages "remotes" and "tools".
# These packages are necessary for the SeuratDisk installation and the RDS -> H5AD file format conversion.

# Load Seurat
library(Seurat)

# Install remotes to the R subdirectory of home
if (!requireNamespace("remotes", quietly = TRUE)) {
    install.packages("remotes", lib="/home/jovyan/R")
}

# Install tools to the R subdirectory of home
install.packages("tools", lib="/home/jovyan/R")

# Load remotes from the R subdirectory of home
library(remotes, lib.loc="/home/jovyan/R")

# Load tools from the R subdirectory of home
library(tools, lib.loc="/home/jovyan/R")

In [ ]:
%%R

# NOTE: SeuratDisk is not currently available on CRAN and, for this version of GitHub, can only be remote-installed via GitHub.

# Install SeuratDisk from GitHub to the R subdirectory of home, upgrade any dependencies
remotes::install_github("mojaveazure/seurat-disk", lib="/home/jovyan/R", upgrade="always")

In [1]:
import os
import requests
import gp
import genepattern
from IPython.display import IFrame
@genepattern.build_ui(name="Write Files", parameters={
    "rdsFile": {
        "name": "RDS file:",
        "description": "clustered RDS file produced by the Seurat.Clustering module",
        "type": "file",
        "kinds": ["rds"]
    },
    "height":{"default":850, "hide":True},
    "width":{"default":850, "hide":True},
    "output_var": {
        "default":"output_var",
        "hide": True
    }
})

def writeFile(rdsFile, height, width):
    job_widget = nbtools.UIOutput(status="Getting file from the GenePattern server...")
    display(job_widget)
    r = gp.GPFile(genepattern.session.get(0), rdsFile)
    basenameRDS =os.path.basename(rdsFile)
    resp = requests.get(rdsFile, headers={
        'Authorization': r.server_data.authorization_header(), 
        'User-Agent': 'GenePatternRest'})
    with open(basenameRDS, "wb") as r:
        r.write(resp.content)
    r.close()
    
    job_widget.status = basenameRDS + ' successfully written to the same folder as this notebook!'
    return


UIBuilder(function_import='nbtools.tool(id="Write Files", origin="Notebook").function_or_method', name='Write …

In [3]:
%%r_build_ui { "name": "R File Conversion", "description": "Run to convert RDS file into python format", "parameters":{"rdsFile": {"name": "RDS file","description": "clustered RDS file produced by the Seurat.Clustering module","type": "file","kinds": ["rds"]},"output_var": {"default":"output_var", "hide": True}}}

# Used Conos to use SeuratDisk

library(Seurat)
library(SeuratDisk)
library(tools)
rds_conversion <- function(rdsFile) {
    
    file_name <- rdsFile
    
    base_name <- basename(file_name)
    # Get name of clustered RDS file without extension, used to name the dropdown_data.txt file
    file_name_without_extension <- file_path_sans_ext(base_name)
    
    # Read in Seurat Clustered object
    seurat_clustered_dataset.object <- readRDS(file = file_name)
    
    # Create file name for h5Seurat object
    h5seurat_name <-paste0(file_name_without_extension, ".h5Seurat")
    
    # Quick fix to avoid conversion changing metadata cols (source: https://github.com/mojaveazure/seurat-disk/issues/23)
    i <- sapply(seurat_clustered_dataset.object@meta.data, is.factor)
    seurat_clustered_dataset.object@meta.data[i] <- lapply(seurat_clustered_dataset.object@meta.data[i], as.character)
    
    # RDS -> H5AD conversion
    SaveH5Seurat(seurat_clustered_dataset.object, filename = h5seurat_name, overwrite = TRUE)
    Convert(h5seurat_name, dest = "h5ad", overwrite = TRUE)
    
    # Extract list of metadata names from Seurat Clustered object
    meta_data_names <- as.list(colnames(seurat_clustered_dataset.object@meta.data))

    # Use grep to find indices of strings matching SeuratClustering iterations
    matching_indices <- grep("snn", meta_data_names)
    # Extract those matching strings from the list
    snn_strings <- meta_data_names[matching_indices]
    # Move those strings to beginning of list (by removing and adding to front)
    meta_data_names <- meta_data_names[!(meta_data_names %in% c("seurat_clusters", snn_strings))]
    meta_data_names_ordered <- c("seurat_clusters", snn_strings, meta_data_names)
    # Specify the file path
    output_file <-paste0(file_name_without_extension, "_dropdown_data.txt")

    # Convert the list to character vector and write to the file
    writeLines(unlist(meta_data_names_ordered), output_file)
    return
}
rds_conversion(rdsFile)


UsageError: Cell magic `%%r_build_ui` not found.


In [4]:
# Import packages
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
from plotly import subplots
from plotly.tools import make_subplots
from plotly import graph_objs as go
import plotly.express as px
import os
import warnings
import sys
from IPython.display import HTML, display

# Prevent printing any errors to console
warnings.filterwarnings('ignore')

# Get file names in the current working directory
curr_dir = os.getcwd()
in_directory = os.listdir(curr_dir)
# Filter out only the files (excluding directories)
files = [file for file in in_directory if os.path.isfile(os.path.join(curr_dir, file))]

content_list = []

# Iterate through the list of files to find text file containing metadata columns, used to populate the corresponding dropdown
for file_name in files:
    if file_name.endswith("_dropdown_data.txt"):
        file_path = os.path.join(os.getcwd(), file_name)
        try:
            with open(file_path, 'r') as file:
                # Read in file contents, line by line
                content_list = file.read().splitlines()
        except FileNotFoundError:
            print(f"Missing a file required to run this cell: {file_name}")

color_scale_options = ["categorical", "continuous"]
file_options = [file for file in os.listdir(os.getcwd()) if file.endswith(".h5ad")]

def determine_coloring(umap_df, target_var, color_scale):
    """ Determine categorical vs continuous coloring, in case user's choice needs to be overridden"""
    # if Categorical coloring & large # of unique numerical values (e.g. >= 40), visualizing is not advisable
    if np.unique(umap_df[target_var]).size >= 40 and color_scale == "categorical":
        try:
            umap_df[target_var] = umap_df[target_var].apply(pd.to_numeric)
        except ValueError: # if values are non-numerical, plot somewhat confusing categorical graph
            display(HTML("<p class='alert alert-warning'>WARNING: The selected data has a large number of unique non-numeric entries, and thus cannot be visualized effectively.</p>"))
        else: # default to continuous, if values are numerical
            color_scale = "continuous"
            display(HTML("<p class='alert alert-warning'>WARNING: The selected data contains a large number of unique values. The plot coloring was changed to continuous for better interpretability.</p>"))
    return umap_df

    

@genepattern.build_ui(name="Visualize Clusters", description= "Display UMAP cluster results", collapse=False, parameters={
    "clustered_file":{"type": "file",
                      "description": "clustered H5AD file produced by the R File Conversion Cell (following Seurat.Clustering module)",
                      "choices": {file_name: file_name for file_name in file_options},
                      "kinds": ["h5ad"], 
                      "optional": False
    },
    "selected_column": {
        "type": "choice",
        "description": "selected metadata column from Seurat object",
        "choices": {col: col for col in content_list},
        "optional": True
    },
    "target_gene":{
        "type": "text",
        "description": "Ensembl Gene ID of target gene for gene expression plot",
        "optional": True
    },
    "color_scale": {
        "type": "choice",
        "description": "continuous/quantitative variable (gradient coloring) or categorical variable (discrete coloring)",
        "choices": {choice: choice for choice in color_scale_options}
    },
    "output_var": {
        "hide": True
    }
})

def plot_umap(clustered_file, selected_column, target_gene, color_scale):
    """Show interactive UMAP plot, colored to represent either one of Seurat metadata columns or the expression level of a specified gene

    Parameters:
    clustered_file (str): Name of H5AD file containing AnnData object, which has gene expression data and UMAP loadings + clustering data from Seurat.Clustering
    selected_column (str): Selected metadata column (loaded into UI builder cell using "_dropdown_data.txt" file)
    target_gene(str): Gene for which expression data is to be shown
    color_scale(str): either categorical or continuous

    Returns:
    None, displays interactive UMAP plot

   """
    # Check for file format compatibility
    if clustered_file.endswith("h5ad") == False:
        display(HTML("<p class='alert alert-warning'>ERROR: Selected file must be of type: h5ad</p>"))
    
    is_gene = False # boolean to store whether UMAP plot is for visualizing metadata or gene expression level
    target_var = '' # var to store target variable name (gene or col)
    
    # Either a metadata column or target gene is to be selected
    if str(target_gene) == '' and str(selected_column) == '': # no parameter selected
        display(HTML("<p class='alert alert-warning'>ERROR: select an input parameter</p>"))
        return
    elif str(target_gene) != '' and str(selected_column) != '': # two parameters selected
        display(HTML("<p class='alert alert-warning'>ERROR: select either metadata column or gene as input parameter</p>"))
        return
    elif str(selected_column) == '': # if metadata column not selected -> gene expression plot
        is_gene = True
        target_var = target_gene
    else: # metadata column selected
        target_var = selected_column
    
    # Identify cluster-colored plots for later step
    cluster_coloring = False
    if "RNA_snn_res" in selected_column or selected_column == "seurat_clusters":
        cluster_coloring = True
    
    # Read in clustered H5AD object
    adata = ad.read_h5ad(clustered_file)

    # Find UMAP in annotation of observations (allowing for some deviation in naming) -- as a precaution in case of change in naming convention
    axis_array_keys = list(adata.obsm)
    umap_key = ''
    for key in axis_array_keys:
        if 'umap' in key.lower():
            umap_key = key
    
    # Access the UMAP coordinates and Seurat clusters from the AnnData object
    umap_coords = adata.obsm[umap_key]
    cluster_assignments = adata.obs["seurat_clusters"]
    
    unique_clusters = np.unique(cluster_assignments)
    cluster_cell_counts = {cluster: np.sum(cluster_assignments == cluster) for cluster in unique_clusters}
    
    # Create a DataFrame with UMAP coordinates and gene expression values
    umap_df = pd.DataFrame(umap_coords, columns=['UMAP1', 'UMAP2'])
    
    # For UMAP plots colored by gene expression:
    if is_gene:
        try: # Access gene expression data, assuming storage in .X attribute
            gene_expression = adata[:, target_var].X 
        except KeyError: # if gene cannot be identified in gene expression matrix
            display(HTML("<p class='alert alert-warning'>ERROR: The gene expression matrix in the selected file does not contain data for the given gene.</p>"))
            return
        
        # Identify the clusters and the number of cells ine ach cluster
        clusters = [cluster_assignments.iloc[entry_index] for entry_index in range(len(umap_coords))]
        cells_in_cluster = [cluster_cell_counts[cluster] for cluster in clusters]

        # Add gene expression data to dataframe
        umap_df[target_var] = gene_expression
        
        # Override color scheme if necessary
        umap_df = determine_coloring(umap_df, target_var, color_scale)
        
        # Create a scatter plot colored by the gene expression values
        fig = px.scatter(umap_df, x='UMAP1', y='UMAP2', color=target_var,
                        hover_data={'UMAP1': False, 'UMAP2': False,
                                    'UMAP cluster': clusters,
                                     'Number of cells in cluster': cells_in_cluster
                                    })
        fig.update_layout(title=f"UMAP Scatterplot Colored by Expression of {target_var} Gene", title_x = 0.5)
    
    # UMAP plot colored by metadata column
    else:
        # Access specified metadata column
        selected_col = adata.obs[target_var] 
        
        # Loop through to map together cluster assignments and UMAP coordinates through UMAP indices
        selected_col_looped = {}
        cluster_assignments_looped = {}
        umap_indices = []
        for entry_index in range(len(umap_coords)):
            umap_coordinates_entry = tuple(umap_coords[entry_index])
        
            selected_col_looped[umap_coordinates_entry] = selected_col.iloc[entry_index]
            cluster_assignments_looped[umap_coordinates_entry] = cluster_assignments.iloc[entry_index]
        
            umap_indices.append(umap_coordinates_entry)
    
        # Add information to dataframe
        umap_df["indices"] = umap_indices
        umap_df["seurat_clusters"] = umap_df["indices"].map(cluster_assignments_looped)
        umap_df["cells_in_cluster"] = umap_df["seurat_clusters"].map(cluster_cell_counts)
        if target_var not in umap_df:
            umap_df[target_var] = umap_df["indices"].map(selected_col_looped)
  
        # Override color scheme if necessary
        umap_df = determine_coloring(umap_df, target_var, color_scale)
    
        # Sort UMAP in order of selected column (in order for legend to reflect order)
        umap_df = umap_df.sort_values(by=[target_var], ignore_index=True)
        
        # Change data types for plotly to plot continuous/categorical color scheme accordingly
        if color_scale == "categorical":
            umap_df[target_var] = umap_df[target_var].apply(str)
        elif color_scale == "continuous":
            umap_df[target_var] = umap_df[target_var].apply(pd.to_numeric)
        
        # If cluster coloring, show # of cells per cluster; if not, then omit from hover data
        if cluster_coloring:
            fig = px.scatter(umap_df, x='UMAP1', y='UMAP2', color=target_var,
                    hover_data={'UMAP1': False, 'UMAP2': False,
                                selected_column:True, "seurat_clusters": True, "cells_in_cluster":True
                                })
        else:
            fig = px.scatter(umap_df, x='UMAP1', y='UMAP2', color=target_var,
                    hover_data={'UMAP1': False, 'UMAP2': False,
                                selected_column:True, "seurat_clusters": True, "cells_in_cluster":False
                                })        
        
        fig.update_layout(title=f"UMAP Scatterplot ({target_var} clustering)", title_x = 0.5)

    # Customize to adjust size, adjust margins for axis labels
    fig.update_layout(width=750, height=750, margin=dict(l=65, r=65, t=100, b=65))
    # Customize axes to hide tick marks, maintain scale ratio
    fig.update_xaxes(tickfont=dict(color="rgba(0,0,0,0)"))
    fig.update_yaxes(tickfont=dict(color="rgba(0,0,0,0)"),scaleanchor="x",scaleratio=1)
    
    fig.show()

UIBuilder(collapse=False, description='Display UMAP cluster results', function_import='nbtools.tool(id="Visual…